In [0]:
from pyspark.sql import SparkSession
import pandas as pd
from azure.storage.blob import BlobServiceClient
import json

# Conexión a Azure Blob Storage y descarga del archivo JSON
connection_string = "*************************************"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_name = "********"
blob_name = "cluster_umap_hdbscan_sbert_cls_topics_normalized.json"

blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
downloaded_blob = blob_client.download_blob().readall()
articles_data = json.loads(downloaded_blob)

extracted_data = [{"clusterid": item["idCluster"], "contentId": item["contentId"]} for item in articles_data]
df = pd.DataFrame(extracted_data)

if not df.empty:
    print(df.head())
else:
    print("El archivo JSON está vacío.")

# Crear una sesión de Spark
spark = SparkSession.builder.getOrCreate()
spark_df = spark.createDataFrame(df)

# Parámetros de conexión JDBC
jdbcHostname = "*********************"
jdbcPort = 1****
jdbcDatabase = "*******"
jdbcUsername = "*******"
jdbcPassword = "*******"
jdbcUrl = f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};databaseName={jdbcDatabase}"

connectionProperties = {
    "user": jdbcUsername,
    "password": jdbcPassword,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

# Define the table name
table_name = "cluster_umap_hdbscan_sbert_cls_topics_normalized"

# Escribir los datos en la tabla SQL utilizando JDBC
spark_df.write.jdbc(url=jdbcUrl, table=table_name, mode='append', properties=connectionProperties)

print(f"Datos insertados en la tabla {table_name} en la base de datos.")

clusterid contentId
0 5 4685429
1 2 4724753
2 4 4768899
3 3 4866215
4 2 4866297
Datos insertados en la tabla cluster_umap_hdbscan_sbert_cls_topics_normalized en la base de datos.